In [1]:
# Import nltk libraries for tokenizing
import nltk
from nltk.tokenize import sent_tokenize

In [20]:
# import torch
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [3]:
# import sys

In [1]:
# print(sys.path)

In [2]:
#Downloaded model from Bert-as-service 
BERT_MODEL = 'uncased_L-12_H-768_A-12'

In [3]:
#Initiating BERT client
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

array([[ 0.13186471,  0.32404095, -0.8270434 , ..., -0.37119603,
        -0.39250115, -0.3172188 ],
       [ 0.2487349 , -0.12334448, -0.38933873, ..., -0.4475626 ,
        -0.5591361 , -0.11345163],
       [ 0.2862733 , -0.18580145, -0.30906862, ..., -0.2959366 ,
        -0.39310566,  0.07640228]], dtype=float32)

In [3]:
#One time download
# nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [42]:
#Function to read text data from txt files
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.read()
    sentences = sent_tokenize(filedata)
#     sentences = [y for x in sentences for y in x] # flatten list    
    return sentences

In [43]:
#As an example I have take Wikipedia plot for Avengers End game. Although this itself is a concise summary of the plot, I wanted to run it for fun
sentences = read_article('avengers_inf_synopsis.txt')
# file_name = 'got_s08e03_full_text.txt'

In [44]:
#Since the model we downlaod is uncased
clean_sentences = sentences
sentences = [s.lower() for s in clean_sentences]

In [45]:
sentences

["thanos and his children - proxima midnight, ebony maw, corvus glaive and cull obsidian - have attacked the asgardian ship in search of the space stone, which is housed in the tesseract that loki had stolen before asgard's destruction.",
 "with thanos already possessing the power stone after decimating xandar, thanos' army swiftly defeats the asgardians.",
 "after a futile counter-attack from the hulk, loki offers the tesseract to thanos in exchange for thor's life - only to be killed himself when thanos anticipates loki's attempt to betray and kill him.",
 'moments before glaive kills him, heimdall uses the power of the bifrost to send hulk to earth.',
 'thanos and his children then depart just as the ship self-destructs, leaving a mourning thor behind and stranded in space.',
 "hulk crash lands at the sanctum sanctorum and is reverted back to bruce banner, who informs stephen strange and wong about thanos' impending arrival.",
 'strange and banner retrieve tony stark and brief him a

In [46]:
#Getting the BERT encodings for all the sentences
got_bert_encoded = bc.encode(sentences)

C:\Users\Sandeep\Anaconda3\lib\site-packages\bert_serving\client\__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=25" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [18]:
# from nltk.cluster import KMeansClusterer
# NUM_CLUSTERS=5
# kclusterer = KMeansClusterer(NUM_CLUSTERS, distance = nltk.cluster.util.cosine_distance, repeats=25)
# assigned_clusters = kclusterer.cluster(got_bert_encoded, assign_clusters = True)

In [19]:
# assigned_clusters

In [50]:
# Clustering with Kmeans algorithm
import numpy as np
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
kmeans = kmeans.fit(got_bert_encoded)

In [53]:
from sklearn.metrics import pairwise_distances_argmin_min

avg = []
for j in range(n_clusters):
    idx = np.where(kmeans.labels_ == j)[0]
    avg.append(np.mean(idx))
closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, got_bert_encoded)
ordering = sorted(range(n_clusters), key=lambda k: avg[k])
summary = ' '.join([clean_sentences[closest[idx]] for idx in ordering])

In [54]:
summary

"The three leave to confront Thanos on Titan, where they meet Quill, Drax and Mantis and formulate a plan to remove Thanos' gauntlet. Vision proposes that Maximoff destroy the Stone in his forehead to keep Thanos from retrieving it. Thanos and his Children - Proxima Midnight, Ebony Maw, Corvus Glaive and Cull Obsidian - have attacked the Asgardian ship in search of the Space Stone, which is housed in the Tesseract that Loki had stolen before Asgard's destruction. While discussing their tactics, Strange uses the Time Stone to view alternate futures, telling them he'd seen 14,000,605 of them, with only one where the Avengers win. Former S.H.I.E.L.D."